In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=44990 sha256=c5c2624b8cdfd12ed8ea71c648c74b0be398d80dfa3de6bec7af669b590ec1b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-s80bocti/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import across_subject_nn_performance



In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/NN/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for NN training
epochs = 1000
batch_size = 2
es_patience = 5
model_dict = {'fe_layers':1, 'activation':'tanh'}


nreps = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']

src_subject_id = 1
#for src_subject_id in range(1,nsubjects+1):

results_df = across_subject_nn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                           exclude, score_list, epochs = epochs, batch_size = batch_size, mv = None, permute = False)

results_df['Shuffled'] = False

perm_results_df = across_subject_nn_performance(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, model_dict,\
                                                exclude, score_list, epochs = epochs, batch_size = batch_size, mv =None, permute = True)

perm_results_df['Shuffled'] = True

#concatenate

results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
results_df['Train_Subject'] = src_subject_id

# #save results to file
results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

/content/drive/MyDrive/EMG_gestures/EMG_data/01
Subject 1|Rep 1
Training Model
Evaluate Model on Trained Data
Epochs Trained: 50
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/06
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/07
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/08
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/09
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/10
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/11
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/12
Evaluate Model
Ta

In [22]:
results_df

,Type,Rep,Test_Subject,Epochs,Batch_Size,Train_Loss,Val_Loss,Epochs_Trained,f1_score,accuracy_score,Shuffled,Train_Subject
0,Test,1,2,1000,2,0.341429,0.391498,39.0,0.282535,0.322967,False,1
1,Test,1,3,1000,2,0.341429,0.391498,39.0,0.750780,0.744737,False,1
2,Test,1,4,1000,2,0.341429,0.391498,39.0,0.608257,0.649289,False,1
3,Test,1,5,1000,2,0.341429,0.391498,39.0,0.692859,0.722646,False,1
4,Test,1,6,1000,2,0.341429,0.391498,39.0,0.533075,0.602041,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...
211,Test,3,35,1000,2,1.389361,2.448059,6.0,0.275441,0.306701,True,1
212,Test,3,36,1000,2,1.389361,2.448059,6.0,0.212683,0.245902,True,1
213,Train,1,1,1000,2,1.522811,1.921840,7.0,0.337825,0.366822,True,1
214,Train,2,1,1000,2,1.224642,1.759811,11.0,0.508898,0.516355,True,1


In [23]:
results_df.groupby(['Type','Shuffled']).mean()

Rep  Test_Subject  ...  accuracy_score  Train_Subject
Type  Shuffled                     ...                               
Test  False       2            19  ...        0.599490              1
      True        2            19  ...        0.242696              1
Train False       2             1  ...        0.915109              1
      True        2             1  ...        0.420561              1

[4 rows x 10 columns]

In [ ]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/log_reg_mv/'

nsubjects = 36


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for RNN training
epochs = 40
batch_size = 2


nreps = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']


for src_subject_id in range(1,nsubjects+1):

    results_df = log_reg_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            epochs = epochs, batch_size = batch_size, mv = 5, permute = False,)

    results_df['Shuffled'] = False

    perm_results_df = log_reg_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                            epochs = epochs, batch_size = batch_size, mv = 5, permute = True,)

    perm_results_df['Shuffled'] = True

    #concatenate

    results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
    results_df['Train_Subject'] = src_subject_id

    # #save results to file
    results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
    results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

Streaming output truncated to the last 5000 lines.
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/20
Evaluate Model
(1307, 16)
(439,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/21
Evaluate Model
(1223, 16)
(395,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/22
Evaluate Model
(1205, 16)
(383,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/23
Evaluate Model
(1167, 16)
(444,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/24
Evaluate Model
(1182, 16)
(439,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/25
Evaluate Model
(1181, 16)
(394,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/26
Evaluate Model
(1090, 16)
(404,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/27
Evaluate Model
(1076, 16)
(422,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/28
Evaluate Model
(991, 16)
(332,)
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/29
Evaluate Mode